# Pruebas MultiFADL One-versus-Rest. fetal dataset

In [1]:
import pickle
import json

import pandas as pd
import tensorflow as tf
import numpy as np
from src import utils

from src.MonoFADLModel import MonoFADLModel
from src.MultiFADLModelOvR import MultiFADLModelOvR
from src.NoSelectionModel import NoSelectionModel

# Seed for neural network executions
SEED = 1234
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [2]:
comparative_results = pd.DataFrame(columns=['test_loss', 'test_accuracy', 'test_f1', 'number of selected features', 'selected Features'])

* Preprocesamiento

In [3]:
# Load dataset and preprocess it
# https://www.kaggle.com/datasets/andrewmvd/fetal-health-classification

# Check if stored, otherwise load and store
try:
    fetal = pickle.load(open('data/fetal_health.pkl', 'rb'))
except:
    fetal = pd.read_csv('data/fetal_health.csv')
    fetal = fetal.drop(['severe_decelerations'], axis=1)
    fetal['fetal_health'] = fetal['fetal_health']-1
    
    with open('data/fetal_health.pkl', 'wb') as f:
        pickle.dump(fetal, f)
        
fetal

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,mean_value_of_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.000,0.000,0.000,0.0,73.0,0.5,43.0,2.4,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,1.0
1,132.0,0.006,0.000,0.006,0.003,0.0,17.0,2.1,0.0,10.4,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,0.0
2,133.0,0.003,0.000,0.008,0.003,0.0,16.0,2.1,0.0,13.4,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,0.0
3,134.0,0.003,0.000,0.008,0.003,0.0,16.0,2.4,0.0,23.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,0.0
4,132.0,0.007,0.000,0.008,0.000,0.0,16.0,2.4,0.0,19.9,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,140.0,0.000,0.000,0.007,0.000,0.0,79.0,0.2,25.0,7.2,...,137.0,177.0,4.0,0.0,153.0,150.0,152.0,2.0,0.0,1.0
2122,140.0,0.001,0.000,0.007,0.000,0.0,78.0,0.4,22.0,7.1,...,103.0,169.0,6.0,0.0,152.0,148.0,151.0,3.0,1.0,1.0
2123,140.0,0.001,0.000,0.007,0.000,0.0,79.0,0.4,20.0,6.1,...,103.0,170.0,5.0,0.0,153.0,148.0,152.0,4.0,1.0,1.0
2124,140.0,0.001,0.000,0.006,0.000,0.0,78.0,0.4,27.0,7.0,...,103.0,169.0,6.0,0.0,152.0,147.0,151.0,4.0,1.0,1.0


In [4]:
# Stratify train-val-test split
X_train, X_val, X_test, y_train, y_val, y_test = utils.train_test_split_stratify(df=fetal, target='fetal_health', SEED=1223)

# Standardization of continuous variables
X_train_scaled, X_val_scaled, X_test_scaled = utils.scale_numerical_variables(X_train, X_test, X_val, numerical_variables=X_train.columns.values.tolist())

Class distribution in train:
0.0    1167
1.0     208
2.0     124
Name: fetal_health, dtype: int64
Class distribution in val:
0.0    240
1.0     43
2.0     25
Name: fetal_health, dtype: int64
Class distribution in test:
0.0    248
1.0     44
2.0     27
Name: fetal_health, dtype: int64


In [5]:
X_train_scaled

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,mean_value_of_long_term_variability,histogram_width,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency
975,0.462963,0.000000,0.000000,0.357143,0.466667,0.0,0.270270,0.294118,0.131868,0.120316,0.593220,0.082569,0.387931,0.3750,0.1,0.653543,0.477064,0.550459,0.338290,1.0
1383,0.277778,0.000000,0.000000,0.571429,0.533333,0.0,0.216216,0.176471,0.000000,0.112426,0.361582,0.183486,0.129310,0.1250,0.1,0.425197,0.357798,0.357798,0.078067,1.0
1365,0.481481,0.000000,0.000000,0.285714,0.000000,0.0,0.148649,0.191176,0.000000,0.258383,0.220339,0.513761,0.224138,0.3125,0.0,0.614173,0.577982,0.559633,0.011152,1.0
1461,0.777778,0.000000,0.000000,0.357143,0.000000,0.0,0.513514,0.044118,0.659341,0.096647,0.050847,0.926606,0.353448,0.0625,0.0,0.787402,0.788991,0.761468,0.000000,1.0
1664,0.000000,0.000000,0.000000,0.642857,0.000000,0.0,0.702703,0.058824,0.000000,0.205128,0.096045,0.477064,0.000000,0.0625,0.0,0.401575,0.348624,0.321101,0.000000,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1881,0.592593,0.000000,0.004158,0.714286,0.800000,0.4,0.621622,0.338235,0.000000,0.023669,0.581921,0.045872,0.336207,0.5000,0.1,0.448819,0.275229,0.385321,0.282528,0.5
310,0.740741,0.000000,0.012474,0.000000,0.000000,0.0,0.770270,0.044118,0.494505,0.216963,0.220339,0.614679,0.318966,0.3750,0.0,0.732283,0.706422,0.688073,0.011152,1.0
1670,0.074074,0.000000,0.000000,0.357143,0.000000,0.0,0.702703,0.058824,0.000000,0.236686,0.152542,0.440367,0.051724,0.0625,0.0,0.393701,0.348624,0.321101,0.003717,0.5
1827,0.629630,0.000000,0.002079,0.142857,0.533333,0.0,0.662162,0.161765,0.000000,0.076923,0.508475,0.146789,0.318966,0.4375,0.1,0.645669,0.568807,0.614679,0.089219,1.0


* Noselection results

In [6]:
model1 = NoSelectionModel(
    n_inputs=X_train_scaled.columns.values.shape[0],
    n_class=y_train.unique().shape[0]
)       

model1.fit(
    X_train_scaled,
    y_train,
    X_val_scaled,
    y_val,
    epochs=50
)

model1.model.save('results/fetal_health/fetal_health_NoSelectionModel.h5')
with open('results/fetal_health/fetal_health_NoSelectionModel_history.pkl', 'wb') as f:
    pickle.dump(model1.history, f)

Epoch 1/50
47/47 [==============================] - 1s 4ms/step - loss: 0.5661 - sparse_categorical_accuracy: 0.7792 - val_loss: 0.4020 - val_sparse_categorical_accuracy: 0.8149
Epoch 2/50
47/47 [==============================] - 0s 1ms/step - loss: 0.3518 - sparse_categorical_accuracy: 0.8466 - val_loss: 0.3497 - val_sparse_categorical_accuracy: 0.8442
Epoch 3/50
47/47 [==============================] - 0s 1ms/step - loss: 0.3144 - sparse_categorical_accuracy: 0.8632 - val_loss: 0.3194 - val_sparse_categorical_accuracy: 0.8474
Epoch 4/50
47/47 [==============================] - 0s 998us/step - loss: 0.2912 - sparse_categorical_accuracy: 0.8779 - val_loss: 0.3277 - val_sparse_categorical_accuracy: 0.8734
Epoch 5/50
47/47 [==============================] - 0s 999us/step - loss: 0.2840 - sparse_categorical_accuracy: 0.8753 - val_loss: 0.2881 - val_sparse_categorical_accuracy: 0.8571
Epoch 6/50
47/47 [==============================] - 0s 977us/step - loss: 0.2618 - sparse_categorical_accu

In [7]:
model1.evaluate(X_test_scaled, y_test)

verbose1 = model1.get_verbose()

comparative_results.loc['NoSelection'] = [verbose1['results']['loss'], 
                                          verbose1['results']['accuracy'], 
                                          verbose1['results']['f1'], 
                                          verbose1['selected_features'].shape[0], 
                                          verbose1['selected_features']]
comparative_results

10/10 [==============================] - 0s 645us/step - loss: 0.2322 - sparse_categorical_accuracy: 0.8840


,test_loss,test_accuracy,test_f1,number of selected features,selected Features
NoSelection,0.232203,0.884013,0.880543,20,"[baseline value, accelerations, fetal_movement..."


* MonoFADL results

In [8]:
model2 = MonoFADLModel(
    n_inputs=X_train_scaled.columns.values.shape[0],
    n_class=y_train.unique().shape[0]
)

model2.fit(
    X_train_scaled,
    y_train,
    X_val_scaled,
    y_val,
    epochs=50
)

model2.model.save('results/fetal_health/fetal_health_MonoFADLModel.h5')
with open('results/fetal_health/fetal_health_MonoFADLModel_history.pkl', 'wb') as f:
    pickle.dump(model2.history, f)

Epoch 1/50
47/47 [==============================] - 1s 4ms/step - loss: 0.6546 - sparse_categorical_accuracy: 0.7458 - val_loss: 0.5369 - val_sparse_categorical_accuracy: 0.7792
Epoch 2/50
47/47 [==============================] - 0s 1ms/step - loss: 0.4745 - sparse_categorical_accuracy: 0.7785 - val_loss: 0.4455 - val_sparse_categorical_accuracy: 0.7792
Epoch 3/50
47/47 [==============================] - 0s 1ms/step - loss: 0.4059 - sparse_categorical_accuracy: 0.7785 - val_loss: 0.3891 - val_sparse_categorical_accuracy: 0.7792
Epoch 4/50
47/47 [==============================] - 0s 1ms/step - loss: 0.3501 - sparse_categorical_accuracy: 0.7785 - val_loss: 0.3412 - val_sparse_categorical_accuracy: 0.7792
Epoch 5/50
47/47 [==============================] - 0s 1ms/step - loss: 0.2957 - sparse_categorical_accuracy: 0.7785 - val_loss: 0.2747 - val_sparse_categorical_accuracy: 0.7792
Epoch 6/50
47/47 [==============================] - 0s 1ms/step - loss: 0.2335 - sparse_categorical_accuracy: 

In [9]:
model2.evaluate(X_test_scaled, y_test)

verbose2 = model2.get_verbose()

comparative_results.loc['MonoFADL'] = [verbose2['results']['loss'], 
                                          verbose2['results']['accuracy'], 
                                          verbose2['results']['f1'], 
                                          verbose2['selected_features'].shape[0], 
                                          verbose2['selected_features']]
comparative_results

10/10 [==============================] - 0s 717us/step - loss: -3.3122 - sparse_categorical_accuracy: 0.7774


,test_loss,test_accuracy,test_f1,number of selected features,selected Features
NoSelection,0.232203,0.884013,0.880543,20,"[baseline value, accelerations, fetal_movement..."
MonoFADL,-3.312196,0.777429,0.680079,1,[accelerations]


In [26]:
verbose2['predictionsproba'][:10]

array([[7.3558778e-01, 1.6687723e-01, 9.7534947e-02],
       [9.8716563e-01, 5.8426135e-03, 6.9916393e-03],
       [9.9769717e-01, 7.8001496e-04, 1.5228052e-03],
       [9.3376172e-01, 3.8164821e-02, 2.8073439e-02],
       [5.8649307e-01, 2.6079828e-01, 1.5270860e-01],
       [9.9769717e-01, 7.8001496e-04, 1.5228052e-03],
       [8.4939921e-01, 9.5860809e-02, 5.4739941e-02],
       [9.9769717e-01, 7.8001496e-04, 1.5228052e-03],
       [5.8649307e-01, 2.6079828e-01, 1.5270860e-01],
       [5.8649307e-01, 2.6079828e-01, 1.5270860e-01]], dtype=float32)

* MultiFADL One-versus-Rest results

In [10]:
model3 = MultiFADLModelOvR(
)

model3.fit(
    X_train_scaled,
    y_train,
    X_val_scaled,
    y_val,
    epochs=50
)

for clas, model in model3.models.items():
    model.model.save(f'results/fetal_health/fetal_health_MultiFADLModel_{clas}.h5')    
    with open(f'results/fetal_health/fetal_health_MultiFADLModel_history_{clas}.pkl', 'wb') as f:
        pickle.dump(model.history, f)

--> Training model class 0.0 vs rest
Epoch 1/50
47/47 [==============================] - 1s 4ms/step - loss: 0.5196 - binary_accuracy: 0.7645 - val_loss: 0.4676 - val_binary_accuracy: 0.7792
Epoch 2/50
47/47 [==============================] - 0s 1ms/step - loss: 0.4262 - binary_accuracy: 0.7785 - val_loss: 0.3836 - val_binary_accuracy: 0.7792
Epoch 3/50
47/47 [==============================] - 0s 1ms/step - loss: 0.3448 - binary_accuracy: 0.7785 - val_loss: 0.3036 - val_binary_accuracy: 0.7792
Epoch 4/50
47/47 [==============================] - 0s 1ms/step - loss: 0.2639 - binary_accuracy: 0.7785 - val_loss: 0.2210 - val_binary_accuracy: 0.7792
Epoch 5/50
47/47 [==============================] - 0s 1ms/step - loss: 0.1803 - binary_accuracy: 0.7785 - val_loss: 0.1329 - val_binary_accuracy: 0.7792
Epoch 6/50
47/47 [==============================] - 0s 1ms/step - loss: 0.0915 - binary_accuracy: 0.7785 - val_loss: 0.0449 - val_binary_accuracy: 0.7792
Epoch 7/50
47/47 [=====================

In [11]:
model3.evaluate(X_test_scaled, y_test)

verbose3 = model3.get_verbose()

comparative_results.loc['MultiFADL'] = [verbose3['results']['loss'], 
                                          verbose3['results']['accuracy'], 
                                          verbose3['results']['f1'], 
                                          {clas: verbose3['selected_features'][clas].shape[0] for clas in verbose3['selected_features']}, 
                                          verbose3['selected_features']]

comparative_results

--> Evaluating model class 0.0 vs rest
10/10 [==============================] - 0s 1ms/step - loss: -3.9986 - binary_accuracy: 0.7774
{'loss': -3.9985787868499756, 'accuracy': 0.777429461479187, 'f1': 0.08103850172815691}
--> Evaluating model class 1.0 vs rest
10/10 [==============================] - 0s 691us/step - loss: -4.1314 - binary_accuracy: 0.8621
{'loss': -4.131368160247803, 'accuracy': 0.8620689511299133, 'f1': 0.7982120051085568}
--> Evaluating model class 2.0 vs rest
10/10 [==============================] - 0s 693us/step - loss: -3.9807 - binary_accuracy: 0.9154
{'loss': -3.980703353881836, 'accuracy': 0.9153605103492737, 'f1': 0.8749108558352873}


,test_loss,test_accuracy,test_f1,number of selected features,selected Features
NoSelection,0.232203,0.884013,0.880543,20,"[baseline value, accelerations, fetal_movement..."
MonoFADL,-3.3122,0.777429,0.680079,1,[accelerations]
MultiFADL,"{0.0: -3.9985787868499756, 1.0: -4.13136816024...","{0.0: 0.777429461479187, 1.0: 0.86206895112991...","{0.0: 0.08103850172815691, 1.0: 0.798212005108...","{0.0: 0, 1.0: 0, 2.0: 0, 'global': 0}","{0.0: [], 1.0: [], 2.0: [], 'global': []}"


In [ ]:
comparative_results.to_csv('results/fetal_health/fetal_health_ComparativeResults.csv')

In [28]:
for clase, probs in verbose3['predictionsproba'].items():
    print(f'{clase}: {probs[:10]}')

0.0: [[0.7804307]
 [0.7804307]
 [0.7804307]
 [0.7804307]
 [0.7804307]
 [0.7804307]
 [0.7804307]
 [0.7804307]
 [0.7804307]
 [0.7804307]]
1.0: [[0.1294443]
 [0.1294443]
 [0.1294443]
 [0.1294443]
 [0.1294443]
 [0.1294443]
 [0.1294443]
 [0.1294443]
 [0.1294443]
 [0.1294443]]
2.0: [[0.08220246]
 [0.08220246]
 [0.08220246]
 [0.08220246]
 [0.08220246]
 [0.08220246]
 [0.08220246]
 [0.08220246]
 [0.08220246]
 [0.08220246]]
global: [[0.7804307  0.1294443  0.08220246]
 [0.7804307  0.1294443  0.08220246]
 [0.7804307  0.1294443  0.08220246]
 [0.7804307  0.1294443  0.08220246]
 [0.7804307  0.1294443  0.08220246]
 [0.7804307  0.1294443  0.08220246]
 [0.7804307  0.1294443  0.08220246]
 [0.7804307  0.1294443  0.08220246]
 [0.7804307  0.1294443  0.08220246]
 [0.7804307  0.1294443  0.08220246]]
